######  <span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#c3b235">Recurrent Neural Network (RNN)
<font color=darkblue>
    
######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#d40eea">Import Libraries
<font color=darkblue>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from scipy.spatial.distance import cdist

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import tensorflow_hub as hub 
import tensorflow_datasets as tfds

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0e36ea">Access and split the dataset
<font color=darkblue>

In [4]:
train_data, validation_data, test_data = tfds.load(
    name = "imdb_reviews",
    split=('train[:60%]','train[60%:]', 'test'),
    as_supervised=True
)

Dataset imdb_reviews downloaded and prepared to /home/jayanthikishore/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_example, train_labels = next(iter(train_data.batch(15)))
train_example

<tf.Tensor: shape=(15,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
tf.shape(train_example)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([15], dtype=int32)>

In [7]:
train_labels

<tf.Tensor: shape=(15,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1])>

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0e36ea">Embeddings
<font color=darkblue>

In [8]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"

In [9]:
hub_layer = hub.KerasLayer(embedding, input_shape =[], dtype=tf.string,trainable=True)
hub_layer(train_example[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0e36ea">Built model (simple)
<font color=darkblue>

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0e36ea">Compile model
<font color=darkblue>

In [11]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics =['accuracy']
              )

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0e36ea">Run the model
<font color=darkblue>

In [12]:
history = model.fit(train_data.shuffle(10000).batch(512),epochs=10,validation_data=validation_data.batch(512),verbose=1)

Epoch 1/10
30/30 [==============================] - 16s 511ms/step - loss: 0.6919 - accuracy: 0.4962 - val_loss: 0.6285 - val_accuracy: 0.5625
Epoch 2/10
30/30 [==============================] - 14s 484ms/step - loss: 0.5951 - accuracy: 0.6127 - val_loss: 0.5218 - val_accuracy: 0.7233
Epoch 3/10
30/30 [==============================] - 14s 483ms/step - loss: 0.4593 - accuracy: 0.7734 - val_loss: 0.4134 - val_accuracy: 0.8097
Epoch 4/10
30/30 [==============================] - 15s 485ms/step - loss: 0.3192 - accuracy: 0.8775 - val_loss: 0.3525 - val_accuracy: 0.8446
Epoch 5/10
30/30 [==============================] - 15s 484ms/step - loss: 0.2293 - accuracy: 0.9173 - val_loss: 0.3262 - val_accuracy: 0.8606
Epoch 6/10
30/30 [==============================] - 15s 484ms/step - loss: 0.1676 - accuracy: 0.9477 - val_loss: 0.3139 - val_accuracy: 0.8601
Epoch 7/10
30/30 [==============================] - 14s 484ms/step - loss: 0.1237 - accuracy: 0.9659 - val_loss: 0.3124 - val_accuracy: 0.8675